In [42]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
from keras.models import model_from_json
%pylab inline
import keras.backend as K
import pickle
import os
import dill
import numpy as np
import cv2 as cv

Populating the interactive namespace from numpy and matplotlib


In [7]:
batch_size = 8
input_size = (128,128,3)

# Create dataset

In [2]:
frontal_face_cascade = cv.CascadeClassifier('Script/opencv/data/haarcascades/haarcascade_frontalface_default.xml')
profile_face_cascade = cv.CascadeClassifier('Script/opencv/data/haarcascades/haarcascade_profileface.xml')

In [48]:
def preprocessing(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = np.array(gray, dtype='uint8')
    faces = frontal_face_cascade.detectMultiScale(gray, 1.3, 5)
    profile_faces = profile_face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces)>0:
        for (x,y,w,h) in faces:
            cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0), 2)
            roi_color = img[y:y+h, x:x+w]
            img[:y,:] = 0
            img[y+h:,:] = 0 
            img[:,:x] = 0 
            img[:,x+h:] = 0
            return img
            break
    elif len(profile_faces)>0:
        for (x,y,w,h) in profile_faces:
            cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0), 2)
            roi_color = img[y:y+h, x:x+w]
            img[:y,:] = 0
            img[y+h:,:] = 0 
            img[:,:x] = 0 
            img[:,x+h:] = 0
            return img
            break
    else:
        return img
    
        

In [6]:
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [8]:
data = pd.read_csv('list_attr_celeba.csv')
data.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [9]:
data['target'] = 1*(data.Eyeglasses == 1)


In [11]:
data = data[['image_id', 'target']]
data.head()

,image_id,target
0,000001.jpg,0
1,000002.jpg,0
2,000003.jpg,0
3,000004.jpg,0
4,000005.jpg,0


In [15]:
data.target = data.target.astype('str') 

## ImageGenerator

In [16]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        validation_split=0.25,
        preprocessing_function = preprocessing,
        horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
data.reset_index(inplace=True, drop=True)


In [18]:
data.head()

,image_id,target
0,000001.jpg,0
1,000002.jpg,0
2,000003.jpg,0
3,000004.jpg,0
4,000005.jpg,0


In [19]:
data.target.value_counts()

0    189406
1     13193
Name: target, dtype: int64

In [20]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe=data, directory = 'D:\\Study\\test_ass\\img_align_celeba',x_col='image_id', y_col='target',
        target_size=input_size[:2],
        batch_size=batch_size,class_mode='binary')

Found 202599 images belonging to 2 classes.


## Create and train model

In [28]:
# Создаем последовательную модель
model_3 = Sequential()

model_3.add(Conv2D(75, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_size))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.2))
model_3.add(Conv2D(100, (3, 3), activation='relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.2))
model_3.add(Conv2D(120, kernel_size=(3, 3), input_shape=input_size))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.2))
model_3.add(Flatten())
model_3.add(Dense(128, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(1, activation='sigmoid'))

In [29]:
model_3.compile(loss='binary_crossentropy', optimizer='adam',metrics=[f1])

In [30]:
early_stopping = EarlyStopping(monitor='loss', min_delta=0.005, patience=3)
checkpoint = ModelCheckpoint(filepath='custom_model_best.pkl', verbose=1, save_best_only=True, monitor='loss')

In [32]:
model_3.fit_generator(train_generator,
        steps_per_epoch=int(len(data)*0.75/batch_size),
        epochs=10,
        validation_steps=int(len(data)*0.25/batch_size),
        callbacks= [early_stopping, checkpoint],
        verbose=1,)

Epoch 1/10
18993/18993 [==============================] - 3795s 200ms/step - loss: 0.0795 - f1: 0.3097

Epoch 00001: loss improved from inf to 0.07946, saving model to custom_model_best.pkl
Epoch 2/10
18993/18993 [==============================] - 2329s 123ms/step - loss: 0.0516 - f1: 0.3460

Epoch 00002: loss improved from 0.07946 to 0.05157, saving model to custom_model_best.pkl
Epoch 3/10
18993/18993 [==============================] - 1747s 92ms/step - loss: 0.0485 - f1: 0.3543

Epoch 00003: loss improved from 0.05157 to 0.04846, saving model to custom_model_best.pkl
Epoch 4/10
18993/18993 [==============================] - 1738s 92ms/step - loss: 0.0486 - f1: 0.3562

Epoch 00004: loss did not improve from 0.04846
Epoch 5/10
18993/18993 [==============================] - 1733s 91ms/step - loss: 0.0455 - f1: 0.3587

Epoch 00005: loss improved from 0.04846 to 0.04548, saving model to custom_model_best.pkl
Epoch 6/10
18993/18993 [==============================] - 1737s 91ms/step - loss

In [33]:
model_json = model_3.to_json()
with open("trained_model_3_small_new_preproc.json", "w") as json_file:
    json_file.write(model_json)
model_3.save_weights("trained_model_3_small_new_preproc.h5")
print("Saved model to disk")


Saved model to disk


## Test

In [34]:
# load json and create model
json_file = open('trained_model_3_small_new_preproc.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("trained_model_3_small_new_preproc.h5")
print("Loaded model from disk")


Loaded model from disk


In [25]:
model_3 = loaded_model

In [75]:
%%time
count=0
path = "/with_glasses/"
for img_path in os.listdir(path):
    #cv.imread(path +img_path)
    img = image.load_img(path +img_path, target_size=(128, 128))
    img = np.array(img, dtype='uint8') 
    img = preprocessing(img)
    img = image.img_to_array(img)/255
    score = model_3.predict(img.reshape(1,128,128,3), verbose=0)
    print(img_path, score)

0.jpg [[0.29246017]]
1.jpg [[0.55439717]]
10.jpg [[0.01253394]]
11.jpg [[0.04831697]]
12.jpg [[0.55229425]]
13.jpg [[0.7771438]]
14.jpg [[0.41137168]]
15.jpg [[0.5768598]]
16.jpg [[0.34123558]]
17.jpg [[0.03088481]]
18.jpg [[0.03828774]]
19.jpg [[0.24798936]]
2.jpg [[0.34006563]]
3.jpg [[0.02430972]]
4.jpg [[0.36056817]]
5.jpg [[0.75340384]]
6.jpg [[0.08363865]]
7.jpg [[0.11342459]]
8.jpg [[0.00584127]]
9.jpg [[0.2871128]]
Wall time: 353 ms


In [77]:
%%time
count=0
path = "/without_glasses/"
for img_path in os.listdir(path):
    img = image.load_img(path +img_path, target_size=(128, 128))
    img = np.array(img, dtype='uint8') 
    #plt.imshow(preprocessing(img))
    img = preprocessing(img)
    img = image.img_to_array(img)/255
    score = model_3.predict(img.reshape(1,128,128,3), verbose=0)
    print(img_path, score)

0.jpg [[0.5349534]]
1.jpg [[0.4229087]]
10.jpg [[0.22240402]]
11.jpg [[0.47031617]]
12.jpg [[0.21629159]]
13.jpg [[0.05395916]]
14.jpg [[0.19959496]]
15.jpg [[0.34343395]]
16.jpg [[0.40634093]]
17.jpg [[0.17447405]]
18.jpg [[0.3306078]]
19.jpg [[0.24458745]]
2.jpg [[0.1895986]]
3.jpg [[0.00019223]]
4.jpg [[0.38061193]]
5.jpg [[0.5907279]]
6.jpg [[0.2129275]]
7.jpg [[0.01136056]]
8.jpg [[0.27019623]]
9.jpg [[0.22456285]]
Wall time: 373 ms
